In [1]:
import pandas as pd
fips = pd.read_csv('/Users/kdugg/Downloads/ZIP-COUNTY-FIPS_2010-03.csv')

In [2]:
df = pd.read_csv('/Users/kdugg/Documents/CAINC30__ALL_AREAS_1969_2018.csv', encoding='latin1')\
.drop(99140)\
.drop(99139)\
.drop(99138)
other_cols = df['Description'][:31]

C:\Users\kdugg\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = df[df['GeoName'].str.contains(', ')].reset_index().drop("index", axis = 1)
df = df.drop("Region", axis = 1)\
.drop('IndustryClassification', axis = 1)\
.drop('TableName', axis = 1)\
.drop('GeoName', axis = 1)\
.drop('Unit', axis = 1)\
.drop('Description', axis = 1)\

In [4]:
fips = fips.drop('COUNTYNAME', axis = 1)\
.drop('STATE', axis = 1)\
.drop('CLASSFP', axis = 1)\

In [5]:
fips['STCOUNTYFP'] = fips['STCOUNTYFP'].apply(lambda fip: str(fip).zfill(5))
df['GeoFIPS'] = df['GeoFIPS'].apply(lambda fip: fip[1:-1])
df['GeoFIPS'] = df['GeoFIPS'].apply(lambda fip: fip[1:-1])

In [6]:
df = df.merge(fips, left_on = 'GeoFIPS', right_on = 'STCOUNTYFP')\
.drop('STCOUNTYFP', axis = 1)\
.drop('GeoFIPS', axis = 1)

In [7]:
df['ZIP_CODE_SHORT'] = df['ZIP'].apply(lambda zi: str(zi).zfill(5)[:3])
df = df.drop('ZIP', axis = 1)

In [8]:
import numpy as np
for i in range(1969, 2019):
    x = str(i)
    df[x] = df[x].apply(lambda x: 0 if x == '(NA)' else x)
    df[x] = df[x].astype('int64')
    df[x] = df[x].apply(lambda x: np.nan if x == 0 else x)

In [9]:
df = df.groupby(['ZIP_CODE_SHORT', 'LineCode']).mean()
df = df.reset_index()

In [10]:
df = pd.melt(df, id_vars=['ZIP_CODE_SHORT', 'LineCode'], value_vars=[str(i) for i in range(1969, 2018)])

In [11]:
vals = df['value'].to_numpy()
index = pd.MultiIndex.from_frame(df.drop('value', axis = 1))
multi = pd.DataFrame(vals, index=index, columns=['value'])

In [12]:
unstacked = multi.unstack(1).reset_index()

In [13]:
cols = ['ZIP_CODE_SHORT', 'DATE']
for x in other_cols:
    l = x.strip().split(' ')
    thel = []
    for y in l:
        if '(' in y:
            break
        if '/' in y:
            break
        y = y.replace(',', '').replace('\'', '')
        thel.append(y.upper())
    cols.append('_'.join(thel))

In [14]:
unstacked.columns = cols

In [15]:
unstacked = unstacked[unstacked['DATE'].str.contains('20')].reset_index().drop('index', axis = 1)
unstacked = pd.concat([unstacked]*12, ignore_index=True).sort_values(['ZIP_CODE_SHORT', 'DATE']).reset_index().drop('index', axis = 1)
months = [str(i).zfill(2) for i in range(1,13)]
l = []
for i,x in enumerate(unstacked['DATE']):
    l.append("{}/01/{}".format(months[(i)%12], x))
unstacked['DATE'] = pd.Series(l)

In [17]:
out = unstacked[['ZIP_CODE_SHORT','DATE','PERSONAL_INCOME','POPULATION']]
out.to_parquet('population_income.parquet')